## 准备数据

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [3]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [12]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        # 使用 GlorotNormal 初始化权重，偏置初始化为零
        initializer = tf.initializers.he_normal()
        #initializer = tf.initializers.HeNormal()
        self.l1 = 2048
        self.W1 = tf.Variable(initializer([28 * 28, self.l1]), dtype=tf.float32)  # 权重 W1，形状 [784, self.l1]
        self.b1 = tf.Variable(tf.zeros(self.l1), dtype=tf.float32)
        self.W2 = tf.Variable(initializer([self.l1, 10]), dtype=tf.float32)  # 权重 W2，形状 [self.l1, 10]
        self.b2 = tf.Variable(tf.zeros(10), dtype=tf.float32)
        
    def __call__(self, x, training=True):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        # 输入x的形状为(60000, 28, 28)
        x = tf.reshape(x, [-1, 28*28])
        h1 = tf.nn.relu(x @ self.W1 + self.b1) # 第一层全连接+relu
        if training:
            h1 = tf.nn.dropout(h1, rate=0.2) # dropout
        logits = h1 @ self.W2 + self.b2 # 全连接输出logits
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [5]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x, training=False)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [13]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.4528577 ; accuracy 0.106983334
epoch 1 : loss 2.3942323 ; accuracy 0.11288334
epoch 2 : loss 2.3499475 ; accuracy 0.120516665
epoch 3 : loss 2.3115704 ; accuracy 0.13265
epoch 4 : loss 2.277062 ; accuracy 0.14861667
epoch 5 : loss 2.246268 ; accuracy 0.1687
epoch 6 : loss 2.2182252 ; accuracy 0.18843333
epoch 7 : loss 2.1890917 ; accuracy 0.2119
epoch 8 : loss 2.1644673 ; accuracy 0.23323333
epoch 9 : loss 2.137699 ; accuracy 0.25586668
epoch 10 : loss 2.112909 ; accuracy 0.27835
epoch 11 : loss 2.0890102 ; accuracy 0.29831666
epoch 12 : loss 2.0656009 ; accuracy 0.31935
epoch 13 : loss 2.0407588 ; accuracy 0.3429
epoch 14 : loss 2.0191789 ; accuracy 0.3609
epoch 15 : loss 1.9985219 ; accuracy 0.38266668
epoch 16 : loss 1.9754335 ; accuracy 0.39986667
epoch 17 : loss 1.9547541 ; accuracy 0.41596666
epoch 18 : loss 1.9338078 ; accuracy 0.433
epoch 19 : loss 1.9138193 ; accuracy 0.45011666
epoch 20 : loss 1.894219 ; accuracy 0.46358332
epoch 21 : loss 1.8739188 ; accurac